<a href="https://colab.research.google.com/github/lustraka/data-analyst-portfolio-project-2022/blob/main/cs01_cds_methods/20211203_Transform_CSV_HTML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transform CSV to HTML and Vice Versa
## Goal
The purpose of this pattern is to provide interface for gathering data into structured form through HTML files in GitHub repository. GitHub displays HTML files as source code, but it is possible to open rendered output using prefix:

```
https://htmlpreview.github.io/?
```
> **Note**: To open the link in a new tab, just use `Ctrl`+click

## Objectives
- Export a DataFrame to an HTML file using `BeautifulSoup`, where:
  - Each row is contained in a `<div class='row'>` element.
  - Fields in general are contained  in `<span class='field_name'>` elements divided by `<br\>` element.
  - Fields `term` and `title` are wrapped in a `<h3>` element.
  - The field `url` is wrapped in a `<a href=''>` element.
- Import a DataFrame from an HTML file. The structure of a DataFrame imported from HTML shall be derived from the classes of `<span>` elements in the HTML file.

## Lessons Learned
Using `BeautifulSoup` for extracting data:
  - cs01_cds_methods > [20211202_Search_Web_Resources.ipynb](https://github.com/lustraka/data-analyst-portfolio-project-2022/blob/875001b6ca1896c6a09d7e71a16a2b1bb65da280/cs01_cds_methods/20211202_Search_Web_Resources.ipynb)
  - suitecrm > iim > 20211126-WebScrape-Pytude.ipynb
  - Wrangle_Data > [Compare_CRM_Systems.ipynb](https://github.com/lustraka/Data_Analysis_Workouts/blob/d0585eafc2d5a1439831da275609945e430527ac/Wrangle_Data/Compare_CRM_Systems.ipynb)
  
Using `BeautifulSoup` for reporting data:
  - Tutorials > BeautifulSoup-Compile-Prehled-zmen.ipynb

## Set Up HTML Template

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from datetime import date
import os

In [2]:
# Initialize the template for an export
html_template = """<!DOCTYPE html>
<html><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title></title>
<style>
body { background-color: #ffffff; color: #000000; margin:10px 10px 10px 10px; font-family: Arial, Helvetica, sans-serif; }
h1 { font-size: 1.75em; color: #003199; }
h2 { font-size: 1.25em; color: #003199; }
h3 { font-size: 1em; color: #003199; }
p, li { font-size: 1em; }
.header { font-size: 2em; color: #000099; }
.maincontent { }
.footer { text-align: center; font-size: 0.675em; }
table, th, td { border: 1px solid black; border-collapse: collapse; }
table { width: 18cm; }
th, td { padding: 2px; }
.highligth { background-color: #ffffe0;}
</style>
</head>
<body>
<div class="header"><span></span></div>
<hr/>
<div class="maincontent">
</div>
<hr/>
<div class="footer">Updated: <span class="update"></span>.</div>
</body></html>"""
soup = BeautifulSoup(html_template, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
  </title>
  <style>
   body { background-color: #ffffff; color: #000000; margin:10px 10px 10px 10px; font-family: Arial, Helvetica, sans-serif; }
h1 { font-size: 1.75em; color: #003199; }
h2 { font-size: 1.25em; color: #003199; }
h3 { font-size: 1em; color: #003199; }
p, li { font-size: 1em; }
.header { font-size: 2em; color: #000099; }
.maincontent { }
.footer { text-align: center; font-size: 0.675em; }
table, th, td { border: 1px solid black; border-collapse: collapse; }
table { width: 18cm; }
th, td { padding: 2px; }
.highligth { background-color: #ffffe0;}
  </style>
 </head>
 <body>
  <div class="header">
   <span>
   </span>
  </div>
  <hr/>
  <div class="maincontent">
  </div>
  <hr/>
  <div class="footer">
   Updated:
   <span class="update">
   </span>
   .
  </div>
 </body>
</html>


In [3]:
soup.find(class_='update').string = str(date.today())
print(soup.body.prettify())

<body>
 <div class="header">
  <span>
  </span>
 </div>
 <hr/>
 <div class="maincontent">
 </div>
 <hr/>
 <div class="footer">
  Updated:
  <span class="update">
   2021-12-03
  </span>
  .
 </div>
</body>


In [4]:
# Load an example CSV file
path = 'https://raw.githubusercontent.com/lustraka/data-analyst-portfolio-project-2022/main/data/web_searches/'
df = pd.read_csv(path+'url_review.csv')
df.shape

(23, 6)

## Design Export of a DataFrame
- For each row append  a `<div class='row'>` element.
- For each field append a `<span class='field_name'>` elements followed by `<br\>` element.
- Add special wrapping for `term`, `title`, and `url`.


In [5]:
filename = 'test.html'
def export_html(df, filename):
  """Export the dataframe df to a HTML file called `filename`."""

  soup = BeautifulSoup(html_template, 'html.parser')
  soup.find(class_='update').string = str(date.today())

  # Locate a div maincontent element
  maincontent = soup.find('div', class_='maincontent')

  for _, row in df.iterrows():
    div_row = BeautifulSoup('<div class="row"></div>').div
    
    # Prepare a <h3> element for term and title
    h3_tag = BeautifulSoup('<h3></h3>').h3
    div_row.append(h3_tag)
    h3 = div_row.h3

    # Prepare an <a> element for url
    a_tag = BeautifulSoup('<a></a>').a
    div_row.append(a_tag)
    a =  div_row.a

    for col in df.columns:
      span = BeautifulSoup('<span></span>').span
      span['class'] = f'{col}'
      span.string = str(row[col])
      if col == 'term':
        h3.append(span)
        h3.append(BeautifulSoup('<br/>').br)
      elif col == 'title':
        h3.append(span)
      elif col == 'url':
        a['href'] = row[col]
        # Open link in a new tab
        a['target'] = '_blank'
        # Prevent tabnabbing!!
        a['rel'] = "noopener noreferrer"
        a.append(span)
        div_row.append(BeautifulSoup('<br/>').br)
      else:
        div_row.append(span)
        div_row.append(BeautifulSoup('<br/>').br)
    maincontent.append(div_row)

  with open(filename, 'w') as file:
    file.write(soup.prettify())
  
  return

# Test export to html
export_html(df,filename)

## Design Import of a HTML File
- Load an HTML file from GitHub
- Locate the maincontent
- Iterate over rows to get dictionaries with variables and their values
- Create a new dataframe from a list of dictionaries
- Save the dataframe to the local csv file
- Return the dataframe

In [6]:
import requests
path = 'https://raw.githubusercontent.com/lustraka/data-analyst-portfolio-project-2022/main/data/web_searches/'


In [7]:
def import_html(path):
  """Import an HTML file, save the CSV and return a DataFrame."""

  r = requests.get(path)
  soup = BeautifulSoup(r.content, 'html.parser')
  rows = []
  # Locate the main content
  main = soup.find('div', class_='maincontent')
  # Iterate over rows to get dictionary of elements
  for row in main.find_all('div', class_='row'):
    rows.append({e['class'][0]:e.string.strip() for e in row.find_all('span')})
  
  df = pd.DataFrame(rows)

  # Save the DataFrame as CSV
  df.to_csv(path.split('/')[-1].split('.')[0]+'.csv', index=False)
  print(f'Dataframe with shape {df.shape} saved as CSV.')

  return df

In [8]:
# Test import
df = import_html(path+'test.html')

Dataframe with shape (23, 7) saved as CSV.


In [9]:
df.iloc[0]

term        url Kitchin 2014 JournalOfRegionalScience book...
title       The Data Revolution: Big Data, Open Data, Data...
url         https://www.researchgate.net/publication/30789...
inn                                      www.researchgate.net
accessed                                           2021-12-02
search                                kitchin+data+revolution
abstract    This is a new field added by editing an HTML f...
Name: 0, dtype: object

In [10]:
df

,term,title,url,inn,accessed,search,abstract
0,url Kitchin 2014 JournalOfRegionalScience book...,"The Data Revolution: Big Data, Open Data, Data...",https://www.researchgate.net/publication/30789...,www.researchgate.net,2021-12-02,kitchin+data+revolution,This is a new field added by editing an HTML f...
1,url Kitchin 2014 pdf fragment,[PDF] data revolution - Building a Digital Por...,http://arthistory2015.doingdh.org/wp-content/u...,arthistory2015.doingdh.org,2021-12-02,kitchin+data+revolution,NaN
2,url Kitchin 2021b BarnesNoble item,"A Critical Analysis of Big Data, Open Data and...",https://www.barnesandnoble.com/w/the-data-revo...,www.barnesandnoble.com,2021-12-02,kitchin+data+revolution,NaN
3,url Google Books search04,The Data Revolution - Rob Kitchin - Google Books,https://books.google.com/books/about/The_Data_...,books.google.com,2021-12-02,kitchin+data+revolution,NaN
4,url Google Books search05,The Data Revolution - Rob Kitchin - Google Books,https://books.google.com/books/about/The_Data_...,books.google.com,2021-12-02,kitchin+data+revolution,NaN
5,url Google Books search06,The Data Revolution: A Critical Analysis of Bi...,https://books.google.com/books/about/The_Data_...,books.google.com,2021-12-02,kitchin+data+revolution,NaN
6,url Kitchin 2014 SurveillanceSociety bookreview,"Book review: Kitchin, R. 2014. 'The Data Revol...",https://openresearch.surrey.ac.uk/esploro/outp...,openresearch.surrey.ac.uk,2021-12-02,kitchin+data+revolution,NaN
7,url Kitchin 2021b Booktopia item,The Data Revolution - Booktopia,https://www.booktopia.com.au/the-data-revoluti...,www.booktopia.com.au,2021-12-02,kitchin+data+revolution,NaN
8,url Kitchin 2014 LesComptesRendus bookreview,"Rob Kitchin, The Data Revolution: Big Data, Op...",https://journals.openedition.org/lectures/16521,journals.openedition.org,2021-12-02,kitchin+data+revolution,NaN
9,url Kitchin 2014 EconPaper reviewrecord,Sage Publications. 222+xvii. ISBN: 978-1446287...,https://econpapers.repec.org/RePEc:bla:jregsc:...,econpapers.repec.org,2021-12-02,kitchin+data+revolution,NaN
